In [2]:
!pip install -q ultralytics opencv-python matplotlib pillow tqdm
print("✓ All packages installed successfully!")

✓ All packages installed successfully!


In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("martnborbalpez/coco-annotated-wind-turbine-surface-damage")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Saiga\.cache\kagglehub\datasets\martnborbalpez\coco-annotated-wind-turbine-surface-damage\versions\2


In [1]:
import json
import os
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import glob

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


In [3]:
def convert_coco_to_yolo(coco_json_path, output_label_dir, img_width=586, img_height=371):
    """
    Convert COCO JSON annotations to YOLO format
    YOLO format: <class_id> <x_center> <y_center> <width> <height> (normalized 0-1)
    """
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # Create category mapping (COCO id to YOLO id)
    categories = {cat['id']: idx for idx, cat in enumerate(coco_data['categories'])}
    cat_names = {cat['id']: cat['name'] for cat in coco_data['categories']}
    
    print(f"\n  Categories found:")
    for coco_id, yolo_id in categories.items():
        print(f"    {yolo_id}: {cat_names[coco_id]}")
    
    # Create output directory
    os.makedirs(output_label_dir, exist_ok=True)
    
    # Group annotations by image_id
    image_annotations = {}
    for ann in coco_data['annotations']:
        img_id = ann['image_id']
        if img_id not in image_annotations:
            image_annotations[img_id] = []
        image_annotations[img_id].append(ann)
    
    # Create image id to filename mapping
    image_files = {img['id']: img['file_name'] for img in coco_data['images']}
    
    # Convert annotations
    converted = 0
    for img_id, annotations in tqdm(image_annotations.items(), desc=f"  Converting {coco_json_path.stem}"):
        if img_id not in image_files:
            continue
            
        img_filename = image_files[img_id]
        label_filename = Path(img_filename).stem + '.txt'
        label_path = Path(output_label_dir) / label_filename
        
        with open(label_path, 'w') as f:
            for ann in annotations:
                # Get COCO bbox: [x, y, width, height]
                bbox = ann['bbox']
                category_id = ann['category_id']
                
                # Convert to YOLO format (normalized)
                x_center = (bbox[0] + bbox[2] / 2) / img_width
                y_center = (bbox[1] + bbox[3] / 2) / img_height
                width = bbox[2] / img_width
                height = bbox[3] / img_height
                
                # Get YOLO class id
                yolo_class_id = categories[category_id]
                
                # Write YOLO format line
                f.write(f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
        
        converted += 1
    
    print(f"  ✓ Converted {converted} images")
    return categories, cat_names

print("✓ Conversion function defined!")

✓ Conversion function defined!


In [4]:
print("[STEP 4/7] Setting up YOLO dataset structure...\n")

# Define paths
base_dir = Path('dataset/NordTank_coco')
yolo_dir = Path('dataset/NordTank_yolo')

# Create YOLO directory structure
for split in ['train', 'valid', 'test']:
    (yolo_dir / 'images' / split).mkdir(parents=True, exist_ok=True)
    (yolo_dir / 'labels' / split).mkdir(parents=True, exist_ok=True)

print("✓ Directory structure created!")

[STEP 4/7] Setting up YOLO dataset structure...

✓ Directory structure created!


In [5]:
print("\n[STEP 5/7] Converting COCO to YOLO format...\n")

# Convert train, valid, test splits
splits = {
    'train': base_dir / 'train',
    'valid': base_dir / 'valid', 
    'test': base_dir / 'test'
}

categories = None
cat_names = None

for split_name, split_path in splits.items():
    json_file = split_path / f'{split_name}.json'
    
    if json_file.exists():
        print(f"\nProcessing {split_name} split:")
        
        # Convert annotations
        categories, cat_names = convert_coco_to_yolo(
            json_file,
            yolo_dir / 'labels' / split_name
        )
        
        # Copy images
        print(f"  Copying images...")
        src_images = list(split_path.glob('*.png')) + list(split_path.glob('*.jpg'))
        for img in tqdm(src_images, desc=f"  Copying {split_name} images"):
            dst = yolo_dir / 'images' / split_name / img.name
            if not dst.exists():
                shutil.copy(img, dst)
        
        print(f"  ✓ {split_name} split complete: {len(src_images)} images\n")

print("\n✓ All splits converted successfully!")


[STEP 5/7] Converting COCO to YOLO format...


Processing train split:

  Categories found:
    0: damage
    1: dirt


  Converting train: 100%|██████████| 2395/2395 [00:02<00:00, 1187.77it/s]


  ✓ Converted 2395 images
  Copying images...


  Copying train images: 100%|██████████| 2395/2395 [00:56<00:00, 42.76it/s]


  ✓ train split complete: 2395 images


Processing valid split:

  Categories found:
    0: damage
    1: dirt


...

  ✓ valid split complete: 300 images


Processing test split:

  Categories found:
    0: damage
    1: dirt


  Converting test: 100%|██████████| 300/300 [00:00<00:00, 1391.49it/s]


  ✓ Converted 300 images
  Copying images...


  Copying test images: 100%|██████████| 300/300 [00:06<00:00, 43.69it/s]

  ✓ test split complete: 300 images


✓ All splits converted successfully!


In [6]:
print("\n[STEP 6/7] Creating YOLO configuration file...\n")

# Create data.yaml for YOLO
data_yaml_content = f"""# NordTank Wind Turbine Blade Damage Detection Dataset
path: {yolo_dir.absolute()}  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/valid  # val images (relative to 'path')
test: images/test  # test images (optional)

# Classes
nc: {len(categories)}  # number of classes
names: {list(cat_names.values())}  # class names
"""

yaml_path = Path('nordtank.yaml')
with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"✓ Configuration saved to: {yaml_path}")
print(f"\nDataset Info:")
print(f"  - Classes: {list(cat_names.values())}")
print(f"  - Train images: {len(list((yolo_dir / 'images' / 'train').glob('*')))}")
print(f"  - Valid images: {len(list((yolo_dir / 'images' / 'valid').glob('*')))}")
print(f"  - Test images: {len(list((yolo_dir / 'images' / 'test').glob('*')))}")


[STEP 6/7] Creating YOLO configuration file...

✓ Configuration saved to: nordtank.yaml

Dataset Info:
  - Classes: ['damage', 'dirt']
  - Train images: 2395
  - Valid images: 300
  - Test images: 300


In [3]:
print("\n[STEP 7/7] Training YOLOv8 Object Detection Model...\n")

from ultralytics import YOLO
import torch

# Check CUDA availability
print("🔍 Checking GPU availability...")
if torch.cuda.is_available():
    print(f"✓ GPU detected: {torch.cuda.get_device_name(0)}")
    device = 0
else:
    print("⚠ No GPU detected, using CPU")
    device = 'cpu'

# Load pretrained YOLOv8 model (nano version for faster training)
model = YOLO('yolov8n.pt')
print("✓ Loaded YOLOv8n pretrained model\n")

# Train the model
print("Starting training... This will take some time.\n")
print("=" * 70)

results = model.train(
    data='nordtank.yaml',
    epochs=50,              # Number of training epochs
    imgsz=640,              # Image size
    batch=16,               # Batch size (adjust based on your GPU/RAM)
    device=device,          # Auto-selected device (GPU or CPU)
    project='runs/detect',  # Project directory
    name='nordtank_damage', # Experiment name
    patience=20,            # Early stopping patience
    save=True,              # Save checkpoints
    plots=True,             # Save plots
    verbose=True,           # Verbose output
    workers=4,              # Number of worker threads
    pretrained=True,        # Use pretrained weights
)

print("\n" + "=" * 70)
print("✓ Training completed!")


[STEP 7/7] Training YOLOv8 Object Detection Model...

🔍 Checking GPU availability...
⚠ No GPU detected, using CPU
✓ Loaded YOLOv8n pretrained model

Starting training... This will take some time.

New https://pypi.org/project/ultralytics/8.3.212 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.199  Python-3.13.5 torch-2.8.0+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=nordtank.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_

In [6]:
print("\n[STEP 8] Evaluating model on validation set...\n")

# Validate the model
metrics = model.val()

# Print metrics
print("\n" + "=" * 70)
print("📊 VALIDATION METRICS")
print("=" * 70)
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP75:     {metrics.box.map75:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")
print("=" * 70)


[STEP 8] Evaluating model on validation set...

Ultralytics 8.3.199  Python-3.13.5 torch-2.8.0+cpu CPU (12th Gen Intel Core(TM) i5-1240P)
val: Fast image access  (ping: 0.10.1 ms, read: 282.1175.5 MB/s, size: 180.2 KB)
val: Scanning C:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\labels\valid.cache... 300 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 300/300 80.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 19/19 1.1it/s 17.5s0.9s
                   all        300        864      0.867      0.762      0.822      0.594
                damage        236        786      0.805      0.613      0.737      0.424
                  dirt         76         78      0.928       0.91      0.907      0.763
Speed: 0.6ms preprocess, 48.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to C:\Users\Saiga\Music\ml project\runs\detect\nordtank_damage43

📊 VALIDATION METRICS
mAP50-95:  0.5936
mAP50:    

In [8]:
# STEP 9: Run Predictions on Test Set (FIXED)
# ---------------------------------------------------------------------------
print("\n[STEP 9] Running predictions on test set...\n")

from pathlib import Path
from ultralytics import YOLO

# Define paths (these should match your dataset structure)
yolo_dir = Path('dataset/NordTank_yolo')  # Your YOLO dataset directory
model_path = 'runs/detect/nordtank_damage4/weights/best.pt'  # Best model from training

# Load the trained model
model = YOLO(model_path)
print("✓ Loaded trained model\n")

# Run inference on test images
test_img_dir = yolo_dir / 'images' / 'test'

if test_img_dir.exists() and list(test_img_dir.glob('*')):
    print(f"Found {len(list(test_img_dir.glob('*')))} test images")
    
    results = model.predict(
        source=str(test_img_dir),
        save=True,
        project='runs/detect',
        name='test_predictions',
        conf=0.25,  # Confidence threshold
        iou=0.45,   # NMS IOU threshold
        show_labels=True,
        show_conf=True,
        line_width=2,
        save_txt=True,  # Save results as text files too
        save_conf=True  # Save confidence scores
    )
    print(f"✓ Predictions saved to: runs/detect/test_predictions/")
    print(f"  - Images with bounding boxes: runs/detect/test_predictions/")
    print(f"  - Text results: runs/detect/test_predictions/labels/")
else:
    print("⚠ No test images found")
    print(f"  Expected path: {test_img_dir}")


[STEP 9] Running predictions on test set...

✓ Loaded trained model

Found 300 test images

image 1/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0006_02_04.png: 416x640 2 damages, 84.8ms
image 2/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0021_03_03.png: 416x640 5 damages, 62.7ms
image 3/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0022_03_03.png: 416x640 15 damages, 65.8ms
image 4/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0022_03_09.png: 416x640 1 damage, 66.1ms
image 5/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0024_03_02.png: 416x640 2 damages, 64.0ms
image 6/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0024_04_08.png: 416x640 1 damage, 62.6ms
image 7/300 c:\Users\Saiga\Music\ml project\dataset\NordTank_yolo\images\test\DJI_0025_04_03.png: 416x640 1 damage, 68.2ms
image 8/300 c:\Users\Saiga\Music\ml proje

In [11]:
# STEP 10: Display Training Results (FIXED)
# ---------------------------------------------------------------------------
print("\n[STEP 10] Displaying results...\n")

from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

# CORRECTED PATHS based on your training output
results_dir = Path('runs/detect/nordtank_damage4')

# Display training results plot
results_img = results_dir / 'results.png'
if results_img.exists():
    img = Image.open(results_img)
    plt.figure(figsize=(16, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Training Results - Loss Curves & Metrics', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    print("✓ Training curves displayed above")
else:
    print(f"⚠ Results plot not found at: {results_img}")
    print("  This is normal if plots=True wasn't enabled or file didn't generate")

# Display confusion matrix
confusion_matrix = results_dir / 'confusion_matrix_normalized.png'
if confusion_matrix.exists():
    img = Image.open(confusion_matrix)
    plt.figure(figsize=(10, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    print("✓ Confusion matrix displayed above")
else:
    print(f"⚠ Confusion matrix not found at: {confusion_matrix}")
    print("  This generates during validation with multiple classes")

# Display validation labels/correctness plot
labels_plot = results_dir / 'labels.jpg'
if labels_plot.exists():
    img = Image.open(labels_plot)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Validation Labels Distribution', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    print("✓ Labels distribution displayed above")

# List all available result files
print(f"\n📁 Available result files in {results_dir}:")
for file in results_dir.glob('*'):
    if file.is_file():
        print(f"  - {file.name}")


[STEP 10] Displaying results...



<Figure size 1600x1000 with 1 Axes>

✓ Training curves displayed above


<Figure size 1000x800 with 1 Axes>

✓ Confusion matrix displayed above


<Figure size 1200x800 with 1 Axes>

✓ Labels distribution displayed above

📁 Available result files in runs\detect\nordtank_damage4:
  - args.yaml
  - BoxF1_curve.png
  - BoxPR_curve.png
  - BoxP_curve.png
  - BoxR_curve.png
  - confusion_matrix.png
  - confusion_matrix_normalized.png
  - labels.jpg
  - results.csv
  - results.png
  - train_batch0.jpg
  - train_batch1.jpg
  - train_batch2.jpg
  - train_batch6000.jpg
  - train_batch6001.jpg
  - train_batch6002.jpg
  - val_batch0_labels.jpg
  - val_batch0_pred.jpg
  - val_batch1_labels.jpg
  - val_batch1_pred.jpg
  - val_batch2_labels.jpg
  - val_batch2_pred.jpg


In [19]:
import os
from pathlib import Path

# Check runs/detect directory structure
detect_dir = Path('runs/detect')
print(f"📁 Contents of runs/detect/:")
print("=" * 50)

if detect_dir.exists():
    for item in sorted(detect_dir.iterdir()):
        if item.is_dir():
            print(f"📂 {item.name}/")
            # Check if this might be the predictions folder
            if 'test' in item.name.lower() or 'pred' in item.name.lower():
                sub_items = list(item.glob('*'))
                print(f"   └─ {len(sub_items)} files: {[f.name for f in sub_items[:5]]}{'...' if len(sub_items) > 5 else ''}")
        else:
            print(f"📄 {item.name}")
else:
    print("❌ runs/detect/ directory does not exist")

# Also check for any folders with 'test' or 'pred' in the name
print(f"\n🔍 Searching for prediction-related folders:")
for item in sorted(detect_dir.iterdir()):
    if item.is_dir() and ('test' in item.name.lower() or 'pred' in item.name.lower()):
        print(f"🎯 Found potential predictions folder: {item.name}/")
        png_files = list(item.glob('*.png'))
        jpg_files = list(item.glob('*.jpg'))
        print(f"   📊 PNG files: {len(png_files)}")
        print(f"   📊 JPG files: {len(jpg_files)}")

📁 Contents of runs/detect/:
📂 nordtank_damage/
📂 nordtank_damage2/
📂 nordtank_damage3/
📂 nordtank_damage4/
📂 nordtank_damage42/
📂 nordtank_damage43/
📂 nordtank_model/
📂 nordtank_model2/
📂 test_predictions/
   └─ 301 files: ['DJI_0006_02_04.jpg', 'DJI_0021_03_03.jpg', 'DJI_0022_03_03.jpg', 'DJI_0022_03_09.jpg', 'DJI_0024_03_02.jpg']...
📂 test_predictions2/
   └─ 113 files: ['DJI_0006_02_04.jpg', 'DJI_0021_03_03.jpg', 'DJI_0022_03_03.jpg', 'DJI_0022_03_09.jpg', 'DJI_0024_03_02.jpg']...

🔍 Searching for prediction-related folders:
🎯 Found potential predictions folder: test_predictions/
   📊 PNG files: 0
   📊 JPG files: 300
🎯 Found potential predictions folder: test_predictions2/
   📊 PNG files: 0
   📊 JPG files: 112


In [20]:
# Step 11: Display Sample Predictions
print("\n[STEP 11] Displaying sample predictions...\n")

from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

# CORRECTED PATH - matches the prediction output
pred_dir = Path('runs/detect/test_predictions')

if pred_dir.exists():
    pred_images = list(pred_dir.glob('*.jpg'))[:9]  # Get first 9 predictions (FIXED: was *.png, now *.jpg)
    
    if pred_images:
        print(f"Found {len(pred_images)} prediction images")
        
        # Display in 3x3 grid
        fig, axes = plt.subplots(3, 3, figsize=(18, 18))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(pred_images):
            img = Image.open(img_path)
            axes[idx].imshow(img)
            axes[idx].axis('off')
            axes[idx].set_title(f"Prediction {idx+1}: {img_path.stem}", fontsize=10)
        
        # Hide unused subplots
        for idx in range(len(pred_images), 9):
            axes[idx].axis('off')
        
        plt.suptitle('Sample Predictions on Test Images (mAP50: 0.822)', fontsize=16, fontweight='bold', y=0.995)
        plt.tight_layout()
        plt.show()
        print(f"✓ Displayed {len(pred_images)} sample predictions")
        
        # Show prediction details
        print("\n📊 Prediction Details:")
        print(f"  - Confidence threshold: 0.25")
        print(f"  - Images processed: {len(list((Path('runs/detect/test_predictions')).glob('*')))}")
        print(f"  - Text results: {len(list((Path('runs/detect/test_predictions/labels')).glob('*.txt')))} files")
        
    else:
        print("⚠ No prediction images found")
        print(f"  Expected in: {pred_dir}")
else:
    print("⚠ Predictions directory not found")
    print(f"  Run STEP 9 first to generate predictions in: {pred_dir}")


[STEP 11] Displaying sample predictions...

Found 9 prediction images


<Figure size 1800x1800 with 9 Axes>

✓ Displayed 9 sample predictions

📊 Prediction Details:
  - Confidence threshold: 0.25
  - Images processed: 301
  - Text results: 279 files


In [16]:
print("\n[STEP 12] Saving final model...\n")

# Copy best model to root directory
best_model_path = Path('runs/detect/nordtank_damage/weights/best.pt')
final_model_path = Path('nordtank_damage_detection_best.pt')

if best_model_path.exists():
    shutil.copy(best_model_path, final_model_path)
    print(f"✓ Best model saved as: {final_model_path}")
    print(f"  Model size: {final_model_path.stat().st_size / (1024*1024):.2f} MB")
else:
    print("⚠ Best model not found")


[STEP 12] Saving final model...

⚠ Best model not found


In [ ]:
print("\n" + "=" * 70)
print("✨ PROJECT COMPLETE! ✨")
print("=" * 70)
print("\n📦 OUTPUT SUMMARY:")
print("\n1. Trained Model:")
print(f"   - Best model: nordtank_damage_detection_best.pt")
print(f"   - Training results: runs/detect/nordtank_damage/")
print("\n2. Predictions:")
print(f"   - Test predictions: runs/detect/predictions/")
print("\n3. Performance Metrics:")
print(f"   - mAP50-95: {metrics.box.map:.4f}")
print(f"   - mAP50: {metrics.box.map50:.4f}")
print(f"   - Precision: {metrics.box.mp:.4f}")
print(f"   - Recall: {metrics.box.mr:.4f}")
print("\n📝 HOW TO USE THE TRAINED MODEL:")
print("\n```python")
print("from ultralytics import YOLO")
print("")
print("# Load trained model")
print("model = YOLO('nordtank_damage_detection_best.pt')")
print("")
print("# Predict on new images")
print("results = model.predict('your_image.jpg', conf=0.25)")
print("")
print("# Display results")
print("results[0].show()")
print("```")
print("\n" + "=" * 70)
print("🎉 Wind Turbine Blade Damage Detection Model Ready!")
print("=" * 70)

In [6]:
import cv2
from ultralytics import YOLO
from pathlib import Path

# Define paths
model_path = r"C:\Users\Saiga\Music\ml project\runs\detect\nordtank_damage4\weights\best.pt"
image_path = r"C:\Users\Saiga\.cache\kagglehub\datasets\martnborbalpez\coco-annotated-wind-turbine-surface-damage\versions\2\NordTank586x371\test\DJI_0004_03_06.png"

# Load the trained model
print("Loading model...")
model = YOLO(model_path)
print("✓ Model loaded successfully!")

# Run inference
print("Running inference...")
results = model(image_path)

# Get the first result (since we're processing one image)
result = results[0]

# Get the original image
img = cv2.imread(image_path)

# Draw bounding boxes on the image
for box in result.boxes:
    # Get box coordinates (xyxy format)
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    
    # Get confidence and class
    conf = float(box.conf[0])
    cls = int(box.cls[0])
    class_name = model.names[cls]
    
    # Choose color based on class (damage=red, dirt=blue)
    color = (0, 0, 255) if class_name == 'damage' else (255, 0, 0)
    
    # Draw rectangle
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
    
    # Create label with class name and confidence
    label = f"{class_name}: {conf:.2f}"
    
    # Get text size for background rectangle
    (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    
    # Draw background rectangle for text
    cv2.rectangle(img, (x1, y1 - text_height - 5), (x1 + text_width, y1), color, -1)
    
    # Put text
    cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

# Display the image
print(f"✓ Detected {len(result.boxes)} objects")
cv2.imshow('Wind Turbine Damage Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Optional: Save the result
output_path = "detection_result.png"
cv2.imwrite(output_path, img)
print(f"✓ Result saved to: {output_path}")

Loading model...
✓ Model loaded successfully!
Running inference...

image 1/1 C:\Users\Saiga\.cache\kagglehub\datasets\martnborbalpez\coco-annotated-wind-turbine-surface-damage\versions\2\NordTank586x371\test\DJI_0004_03_06.png: 416x640 6 damages, 281.9ms
Speed: 9.6ms preprocess, 281.9ms inference, 29.2ms postprocess per image at shape (1, 3, 416, 640)
✓ Detected 6 objects
✓ Result saved to: detection_result.png


In [9]:
import cv2
from ultralytics import YOLO
from pathlib import Path
import glob

# Define paths
model_path = r"C:\Users\Saiga\Music\ml project\runs\detect\nordtank_damage4\weights\best.pt"
test_images_dir = r"C:\Users\Saiga\.cache\kagglehub\datasets\martnborbalpez\coco-annotated-wind-turbine-surface-damage\versions\2\NordTank586x371\test"

# Load the trained model
print("Loading model...")
model = YOLO(model_path)
print("✓ Model loaded successfully!")
print(f"Model classes: {model.names}")

# Get all PNG images
image_files = sorted(glob.glob(str(Path(test_images_dir) / "*.png")))
print(f"\nFound {len(image_files)} test images")

if len(image_files) == 0:
    print("Error: No PNG images found!")
    exit()

# Optional: Setup video writer to save output
output_path = "test_images_detection.mp4"
first_img = cv2.imread(image_files[0])
height, width = first_img.shape[:2]
fps = 2  # 2 frames per second (adjust as needed)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print(f"Image size: {width}x{height}")
print(f"Playing at {fps} FPS")
print("\nProcessing images... Press 'q' to quit, 'p' to pause.\n")

total_detections = 0
paused = False

for idx, img_path in enumerate(image_files):
    # Read image
    frame = cv2.imread(img_path)
    
    if frame is None:
        print(f"Warning: Could not read {img_path}")
        continue
    
    # Run inference
    results = model(frame, conf=0.25, verbose=False)
    result = results[0]
    
    num_detections = len(result.boxes)
    total_detections += num_detections
    
    # Draw bounding boxes
    for box in result.boxes:
        # Get box coordinates
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        
        # Get confidence and class
        conf = float(box.conf[0])
        cls = int(box.cls[0])
        class_name = model.names[cls]
        
        # Choose color (damage=red, dirt=blue)
        color = (0, 0, 255) if class_name == 'damage' else (255, 0, 0)
        
        # Draw rectangle
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        
        # Create label
        label = f"{class_name}: {conf:.2f}"
        
        # Get text size
        (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        
        # Draw background for text
        cv2.rectangle(frame, (x1, y1 - text_height - 5), (x1 + text_width, y1), color, -1)
        
        # Put text
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Add info overlay
    img_name = Path(img_path).name
    info_text = f"Image {idx+1}/{len(image_files)}: {img_name}"
    cv2.putText(frame, info_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    detection_text = f"Detections: {num_detections}"
    cv2.putText(frame, detection_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    # Write frame to output video
    out.write(frame)
    
    # Display frame
    cv2.imshow('Wind Turbine Test Images - Detection', frame)
    
    # Print progress
    print(f"[{idx+1}/{len(image_files)}] {img_name}: {num_detections} detections")
    
    # Handle key press
    while True:
        key = cv2.waitKey(500) & 0xFF  # Wait 500ms (2 FPS)
        
        if key == ord('q'):
            print("\nStopped by user.")
            break
        elif key == ord('p'):
            paused = not paused
            if paused:
                print("PAUSED - Press 'p' to resume")
            else:
                print("RESUMED")
        
        if not paused or key == ord('p'):
            break
    
    if key == ord('q'):
        break

# Release resources
out.release()
cv2.destroyAllWindows()

print(f"\n{'='*60}")
print(f"SUMMARY:")
print(f"{'='*60}")
print(f"Total images processed: {idx+1}")
print(f"Total detections: {total_detections}")
print(f"Average detections per image: {total_detections/(idx+1):.2f}")
print(f"✓ Output video saved to: {output_path}")

Loading model...
✓ Model loaded successfully!
Model classes: {0: 'damage', 1: 'dirt'}

Found 4041 test images
Image size: 586x371
Playing at 2 FPS

Processing images... Press 'q' to quit, 'p' to pause.

[1/4041] DJI_0004_01_07.png: 0 detections
[2/4041] DJI_0004_02_06.png: 9 detections
[3/4041] DJI_0004_03_06.png: 6 detections
[4/4041] DJI_0004_04_05.png: 1 detections
[5/4041] DJI_0004_05_06.png: 0 detections
[6/4041] DJI_0004_06_06.png: 0 detections
[7/4041] DJI_0004_07_05.png: 0 detections
[8/4041] DJI_0004_07_06.png: 1 detections
[9/4041] DJI_0004_08_05.png: 0 detections
[10/4041] DJI_0005_01_05.png: 0 detections
[11/4041] DJI_0005_03_06.png: 5 detections
[12/4041] DJI_0005_04_05.png: 1 detections
[13/4041] DJI_0005_04_06.png: 0 detections
[14/4041] DJI_0005_04_07.png: 0 detections
[15/4041] DJI_0005_05_04.png: 1 detections
[16/4041] DJI_0005_05_05.png: 0 detections
[17/4041] DJI_0005_06_04.png: 0 detections
[18/4041] DJI_0005_07_04.png: 1 detections
[19/4041] DJI_0005_08_04.png: 0 

KeyboardInterrupt: 